Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import math
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    if data.shape[0] == batch_size:
            print('data shape: ', data.shape)
            print('conv 1 shape: ', conv.shape)
            print('hidden 1 shape: ', hidden.shape)
            print()
            
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    if data.shape[0] == batch_size:
            print('conv 2 shape: ', conv.shape)
            print('hidden 2 shape: ', hidden.shape)
            
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

data shape:  (16, 28, 28, 1)
conv 1 shape:  (16, 14, 14, 16)
hidden 1 shape:  (16, 14, 14, 16)

conv 2 shape:  (16, 7, 7, 16)
hidden 2 shape:  (16, 7, 7, 16)


In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.571789
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.261974
Minibatch accuracy: 12.5%
Validation accuracy: 10.8%
Minibatch loss at step 100: 1.811280
Minibatch accuracy: 43.8%
Validation accuracy: 39.3%
Minibatch loss at step 150: 0.708667
Minibatch accuracy: 87.5%
Validation accuracy: 71.1%
Minibatch loss at step 200: 0.894897
Minibatch accuracy: 68.8%
Validation accuracy: 75.1%
Minibatch loss at step 250: 1.408107
Minibatch accuracy: 56.2%
Validation accuracy: 76.6%
Minibatch loss at step 300: 0.407033
Minibatch accuracy: 87.5%
Validation accuracy: 77.9%
Minibatch loss at step 350: 0.396029
Minibatch accuracy: 93.8%
Validation accuracy: 76.3%
Minibatch loss at step 400: 0.376518
Minibatch accuracy: 87.5%
Validation accuracy: 79.4%
Minibatch loss at step 450: 0.895653
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.759946
Minibatch accuracy: 87.5%
Validation accuracy: 80.2%
Mi

# ---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
###############################################################################
############################################################
######################################
import timeit
default_batch_size = 10
patch_size = 5
depth = 8
depth_2 = 16
num_hidden = 128
num_hidden_2 = 64
num_steps = 3001
print_step = num_steps // 10
w_std = 0.1
########################################
##########################################################
###############################################################################

In [ ]:
# network-1
# 89.6%, 90.4 with max_pool
# 88.3%, 88.9% with conv/strides

graph = tf.Graph()

with graph.as_default():
    keep_prob = tf.placeholder(tf.float32, name='keepProb')
    batch_size = tf.placeholder(tf.int32, name='batch_size')
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(None, image_size, image_size, 
                           num_channels), name='train_data')
    print('data shape', tf_train_dataset.shape)
    if tf_train_dataset.shape[0]:
        print('shape[0]: ', tf_train_dataset.shape[0])
    tf_train_labels = tf.placeholder(tf.float32, 
                        shape=(None, num_labels), name='train_labels')
    print('label shape: ', tf_train_labels.shape)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.

    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=w_std))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth_2], stddev=w_std))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_2, num_hidden], stddev=w_std))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden_2], stddev=w_std))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
    
    layerF_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=w_std))
    layerF_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    print('layer1: ', layer1_weights.shape)
    print('layer2: ', layer2_weights.shape)
    print('layer3: ', layer3_weights.shape)
    print('layer4: ', layer4_weights.shape)
    print('layerF: ', layerF_weights.shape)

  # Model.
    def model(data, keep_prob, batch_size):
        pool = tf.nn.max_pool(data, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='c_1')
        conv = tf.nn.conv2d(pool, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        
        if data.shape[0] == batch_size:
            print('data shape: ', data.shape)
            print('pool 1 shape: ', pool.shape)
            print('conv 1 shape: ', conv.shape)
            print('hidden 1 shape: ', hidden.shape)
            print()
    
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='c_2')
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        
        if data.shape[0] == batch_size:
            print('pool 2 shape: ', pool.shape)
            print('conv 2 shape: ', conv.shape)
            print('hidden 2 shape: ', hidden.shape)
        
        shape = hidden.get_shape().as_list()
        
        if not shape[0]:
            print('reshape none')
            shape[0] = batch_size
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)

        hiddenF = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        hiddenF = tf.nn.dropout(hiddenF, keep_prob)
        
        print('shape: ', shape)
        print('reshape: ', reshape.shape)
        print('hidden shape: ', hidden.shape)
        print('hiddenF shape: ', hiddenF.shape)

        if data.shape[0] == batch_size:
            print('shape: ', shape)
            print('reshape: ', reshape.shape)
            print('hidden shape: ', hidden.shape)
            print('hiddenF shape: ', hiddenF.shape)

        return tf.matmul(hiddenF, layerF_weights) + layerF_biases
  
    # Training computation.
    logits = model(tf_train_dataset, keep_prob, batch_size)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
#     optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 
                                           keep_prob, tf_valid_dataset.shape[0]))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, 
                                          keep_prob, tf_test_dataset.shape[0]))
    
# if pool after
# data shape:  (64, 28, 28, 1)
# conv 1 shape:  (64, 28, 28, 16)
# hidden 1 shape:  (64, 28, 28, 16)
# pool 1 shape:  (64, 14, 14, 16)
# conv 2 shape:  (64, 14, 14, 16)
# hidden 2 shape:  (64, 14, 14, 16)
# pool 2 shape:  (64, 7, 7, 16)

data shape (?, 28, 28, 1)
shape[0]:  ?
label shape:  (?, 10)
layer1:  (5, 5, 1, 8)
layer2:  (5, 5, 8, 16)
layer3:  (784, 128)
layer4:  (128, 64)
layerF:  (64, 10)
reshape none
shape:  [<tf.Tensor 'batch_size:0' shape=<unknown> dtype=int32>, 7, 7, 16]
reshape:  (?, 784)
hidden shape:  (?, 128)
hiddenF shape:  (?, 64)
data shape:  (10000, 28, 28, 1)
pool 1 shape:  (10000, 14, 14, 1)
conv 1 shape:  (10000, 14, 14, 8)
hidden 1 shape:  (10000, 14, 14, 8)

pool 2 shape:  (10000, 7, 7, 8)
conv 2 shape:  (10000, 7, 7, 16)
hidden 2 shape:  (10000, 7, 7, 16)
shape:  [10000, 7, 7, 16]
reshape:  (10000, 784)
hidden shape:  (10000, 128)
hiddenF shape:  (10000, 64)
shape:  [10000, 7, 7, 16]
reshape:  (10000, 784)
hidden shape:  (10000, 128)
hiddenF shape:  (10000, 64)
data shape:  (10000, 28, 28, 1)
pool 1 shape:  (10000, 14, 14, 1)
conv 1 shape:  (10000, 14, 14, 8)
hidden 1 shape:  (10000, 14, 14, 8)

pool 2 shape:  (10000, 7, 7, 8)
conv 2 shape:  (10000, 7, 7, 16)
hidden 2 shape:  (10000, 7, 7, 16

In [ ]:
# run-1
start = timeit.default_timer()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    b_size = step + 1
#     if step >= 500:
#         b_size = 500
#     else:    
#         b_size = step + 1
#     b_size = math.ceil(step/3 + 1)
#     print('batch_size: ', b_size)
    
    offset = (step * b_size) % (train_labels.shape[0] - b_size)
#     print('offset: ', offset)
    
    batch_data = train_dataset[offset:(offset + b_size), :, :, :]
#     print('batch shape: ', batch_data.shape)
    
    batch_labels = train_labels[offset:(offset + b_size), :]
#     print('label shape: ', batch_labels.shape)
    
#     print(type(feed_dict))
#     print(type(tf_train_dataset))
#     print(type(tf_train_labels))
#     print(type(keep_prob))
    feed_dict = {tf_train_dataset : batch_data, 
                 tf_train_labels : batch_labels, keep_prob:0.5, batch_size:b_size}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
#     if (step % print_step == 0):
    if (step % 100 == 0):
      feed_dict['keep_prob'] = 1.0
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval({keep_prob:1.0}), valid_labels))
    
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob:1.0}), test_labels))
  elapsed = (timeit.default_timer() - start)/60
  print('execution time: ', elapsed)
#######################################
#######################################################

Initialized
Minibatch loss at step 0: 2.979654
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 100: 2.293491
Minibatch accuracy: 10.9%
Validation accuracy: 44.4%
Minibatch loss at step 200: 1.784136
Minibatch accuracy: 33.8%
Validation accuracy: 65.9%
Minibatch loss at step 300: 1.200912
Minibatch accuracy: 63.1%
Validation accuracy: 77.6%
Minibatch loss at step 400: 1.073745
Minibatch accuracy: 68.6%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.926365
Minibatch accuracy: 71.7%
Validation accuracy: 81.5%
Minibatch loss at step 600: 0.914414
Minibatch accuracy: 72.4%
Validation accuracy: 82.2%
Minibatch loss at step 700: 0.695515
Minibatch accuracy: 80.5%
Validation accuracy: 83.2%
Minibatch loss at step 800: 0.750654
Minibatch accuracy: 78.7%
Validation accuracy: 83.8%
Minibatch loss at step 900: 0.631735
Minibatch accuracy: 82.6%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 0.608645
Minibatch accuracy: 82.4%
Validation accuracy: 84.9%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [ ]:
###############################################################################
############################################################
######################################
# import timeit
# batch_size = 128
# patch_size = 5
# depth = 6
# depth_2 = 16
# num_hidden = 120
# num_hidden_2 = 84
# num_steps = 2001
# print_step = num_steps // 10
# w_std = 0.1
########################################
##########################################################
###############################################################################

In [ ]:
# network-2
# 89.6%, 90.4 with max_pool
# 88.3%, 88.9% with conv/strides

graph = tf.Graph()

with graph.as_default():

    keep_prob = tf.placeholder(tf.float32, name='keepProb')
    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=w_std))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth_2], stddev=w_std))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth_2]))
    
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth_2, num_hidden], stddev=w_std))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden_2], stddev=w_std))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden_2]))
    
    layerF_weights = tf.Variable(tf.truncated_normal(
      [num_hidden_2, num_labels], stddev=w_std))
    layerF_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
#     layer4_weights = tf.Variable(tf.truncated_normal(
#       [num_hidden, num_labels], stddev=0.1))
#     layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    print('layer1: ', layer1_weights.shape)
    print('layer2: ', layer2_weights.shape)
    print('layer3: ', layer3_weights.shape)
    print('layer4: ', layer4_weights.shape)
    
  # Model.
    def model(data, keep_prob):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='c_1')
        
        if data.shape[0] == batch_size:
            print('data shape: ', data.shape)
            print('conv 1 shape: ', conv.shape)
            print('hidden 1 shape: ', hidden.shape)
            print('pool 1 shape: ', pool.shape)
        
        conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME', name='c_2')
        
        if data.shape[0] == batch_size:
            print('conv 2 shape: ', conv.shape)
            print('hidden 2 shape: ', hidden.shape)
            print('pool 2 shape: ', pool.shape)
            
            
        shape = pool.get_shape().as_list()
        reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        hidden = tf.nn.dropout(hidden, keep_prob)
            
        hiddenF = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        hiddenF = tf.nn.dropout(hiddenF, keep_prob)
        
        if data.shape[0] == batch_size:
            print('shape: ', shape)
            print('reshape: ', reshape.shape)
            print('hidden shape: ', hidden.shape)
            print('hiddenF shape: ', hiddenF.shape)
        
        return tf.matmul(hiddenF, layerF_weights) + layerF_biases
    
    
    
        
#         shape = pool.get_shape().as_list()
#         reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
#         hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
# #         ValueError: Dimensions must be equal, but are 3136 and 784 
# #             for 'MatMul' (op: 'MatMul') with input shapes: [64,3136], [784,128].
#         return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset, keep_prob)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
#     optimizer = tf.train.GradientDescentOptimizer(0.04).minimize(loss)
    optimizer = tf.train.AdamOptimizer().minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, keep_prob))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, keep_prob))

In [ ]:
# run-2
start = timeit.default_timer()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % print_step == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval({keep_prob:1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval({keep_prob:1.0}), test_labels))
  elapsed = timeit.default_timer() - start
  print('execution time: ', elapsed/60)